<a href="https://colab.research.google.com/github/justtuananh/Learn_Huggies_face/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers 
!pip install datasets

In [3]:
from datasets import load_dataset 

raw_datasets = load_dataset("glue","mrpc")
raw_datasets 

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

Kiểm tra dữ liệu tập train


In [5]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0] 

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [9]:
raw_validate_dataset = raw_datasets["validation"]
raw_validate_dataset[0]

{'sentence1': "He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .",
 'sentence2': '" The foodservice pie business does not fit our long-term growth strategy .',
 'label': 1,
 'idx': 9}

In [11]:
raw_validate_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

Tiền xử lí một bộ dữ liệu

In [12]:
from transformers import AutoTokenizer 

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenzed_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenzed_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

In [15]:
inputs = tokenizer("this is the first sentences.", "this is the second sentences")
inputs 

{'input_ids': [101, 1142, 1110, 1103, 1148, 12043, 119, 102, 1142, 1110, 1103, 1248, 12043, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentences',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'sentences',
 '[SEP]']

In [21]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"], 
    raw_datasets["train"]["sentence2"], 
    padding = True, 
    truncation = True, 
)

In [23]:
def tokenize_function(example): 
    return tokenizer(example["sentence1"], example["sentence2"], truncation = True)
    

In [24]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)
tokenized_datasets 

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

hàm chịu trách nhiệm tập hợp các mẫu lại với nhau trong 1 lô gọi là hàm đối chiếu. 

In [25]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in["idx", "sentence1", "sentence2"] }
[len(x) for x in samples["input_ids"]]

[52, 59, 47, 69, 60, 50, 66, 32]

Các mẫu có độ dài khác nhau từ 32-69 nên phải chọn đệm động làm 69 

> Văn bản được thụt lề



In [28]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 69]),
 'token_type_ids': torch.Size([8, 69]),
 'attention_mask': torch.Size([8, 69]),
 'labels': torch.Size([8])}